# Download Libs

In [1]:
#import nltk
#nltk.download('all')

In [2]:
#pip install -U spacy

# Restart Runtime

In [3]:
#!python3 -m spacy download en_core_web_trf

In [4]:
# pip install spacy-transformers -f https://download.pytorch.org/whl/torch_stable.html

# Download Data

In [5]:
#from google.colab import files
#files.upload() #enviar o arquivo kaggle.json

#antes de importar o dataset nós iremos provisionar o local de armazenamento
#Alterar a permissão para evitar o aviso durante a partida da ferramenta Kaggle
# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

!rm ./data/*
!kaggle competitions download -c sentiment-analysis-on-movie-reviews
!unzip ./sentiment-analysis-on-movie-reviews.zip -d ./data/
!rm sentiment-analysis-on-movie-reviews.zip
!unzip ./data/test.tsv.zip -d ./data/
!unzip ./data/train.tsv.zip -d ./data/
!rm ./data/train.tsv.zip
!rm ./data/test.tsv.zip

  0%|                                               | 0.00/1.90M [00:00<?, ?B/s]
100%|██████████████████████████████████████| 1.90M/1.90M [00:00<00:00, 20.7MB/s]
Archive:  ./sentiment-analysis-on-movie-reviews.zip
  inflating: ./data/sampleSubmission.csv  
  inflating: ./data/test.tsv.zip     
  inflating: ./data/train.tsv.zip    
Archive:  ./data/test.tsv.zip
  inflating: ./data/test.tsv         
Archive:  ./data/train.tsv.zip
  inflating: ./data/train.tsv        


# Read Data

In [6]:
import pandas as pd

test = pd.read_csv('./data/test.tsv', sep='\t')
train = pd.read_csv('./data/train.tsv', sep='\t')
# sample_submission = pd.read_csv('./data/sampleSubmission.csv', sep='\t')

In [7]:
train = train.drop_duplicates(subset = "SentenceId")

In [8]:
train

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
63,64,2,"This quiet , introspective and entertaining in...",4
81,82,3,"Even fans of Ismail Merchant 's work , I suspe...",1
116,117,4,A positively thrilling combination of ethnogra...,3
156,157,5,Aggressive self-glorification and a manipulati...,1
...,...,...,...,...
155984,155985,8540,... either you 're willing to go with this cla...,2
155997,155998,8541,"Despite these annoyances , the capable Claybur...",2
156021,156022,8542,-LRB- Tries -RRB- to parody a genre that 's al...,1
156031,156032,8543,The movie 's downfall is to substitute plot fo...,1


# Tokenization

In [9]:
import nltk
import spacy
nlp = spacy.load("en_core_web_trf")

/home/gfmco/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
from nltk.corpus import stopwords
stopwords = [word for word in stopwords.words('english')] + [word for word in nlp.Defaults.stop_words]
stopwords = [word.lower() for word in stopwords]

def tokenize(text):
	doc = nlp(text)
	tokens = [token.orth_.lower() for token in doc if token.is_alpha and (token.orth_.lower() not in stopwords)]
	text_pre = None
	for token in tokens:
		if text_pre is None:
			text_pre = token
		else:
			text_pre = text_pre + ' ' + token
	return text_pre

In [11]:
def lemmatize(text):
	doc = nlp(text)
	tokens = [token.lemma_ for token in doc]
	text_lemma = None
	for token in tokens:
		if text_lemma is None:
			text_lemma = token
		else:
			text_lemma = text_lemma + ' ' + token
	return text_lemma

In [12]:
import nltk
from nltk.stem.porter import *
stemmer = PorterStemmer()

In [13]:
def stematize(text):
	doc = nlp(text)	
	tokens = [stemmer.stem(token.orth_) for token in doc]
	text_stema = None
	for token in tokens:
		if text_stema is None:
			text_stema = token
		else:
			text_stema = text_stema + ' ' + token
	return text_stema

In [14]:
def pre_process(text):
	if text is None:
		return text
	else: 
		return stematize(lemmatize(text))

In [15]:
train

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
63,64,2,"This quiet , introspective and entertaining in...",4
81,82,3,"Even fans of Ismail Merchant 's work , I suspe...",1
116,117,4,A positively thrilling combination of ethnogra...,3
156,157,5,Aggressive self-glorification and a manipulati...,1
...,...,...,...,...
155984,155985,8540,... either you 're willing to go with this cla...,2
155997,155998,8541,"Despite these annoyances , the capable Claybur...",2
156021,156022,8542,-LRB- Tries -RRB- to parody a genre that 's al...,1
156031,156032,8543,The movie 's downfall is to substitute plot fo...,1


In [16]:
train['Phrase'] = train['Phrase'].map(lambda text: pre_process(tokenize(text)))

/home/gfmco/.local/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [37]:
train

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,seri escapad demonstr adag good goos good gand...,1
63,64,2,quiet introspect entertain independ worth seek,4
81,82,3,fan ismail merchant work suspect hard time sit,1
116,117,4,posit thrill combin ethnographi intrigu betray...,3
156,157,5,aggress self glorif manipul whitewash,1
...,...,...,...,...
155984,155985,8540,will claustrophob concept,2
155997,155998,8541,despit annoy capabl clayburgh tambor great job...,2
156021,156022,8542,tri parodi genr joke unit state,1
156031,156032,8543,movi downfal substitut plot person,1


In [42]:
trainProcessed = train
trainProcessed

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,seri escapad demonstr adag good goos good gand...,1
63,64,2,quiet introspect entertain independ worth seek,4
81,82,3,fan ismail merchant work suspect hard time sit,1
116,117,4,posit thrill combin ethnographi intrigu betray...,3
156,157,5,aggress self glorif manipul whitewash,1
...,...,...,...,...
155984,155985,8540,will claustrophob concept,2
155997,155998,8541,despit annoy capabl clayburgh tambor great job...,2
156021,156022,8542,tri parodi genr joke unit state,1
156031,156032,8543,movi downfal substitut plot person,1


In [43]:
trainProcessed.to_csv("./processedData/trainProcessed.csv", index=False)
trainProcessed = pd.read_csv("./processedData/trainProcessed.csv", )

In [69]:
trainProcessed[trainProcessed['Phrase'].notnull()]

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,seri escapad demonstr adag good goos good gand...,1
1,64,2,quiet introspect entertain independ worth seek,4
2,82,3,fan ismail merchant work suspect hard time sit,1
3,117,4,posit thrill combin ethnographi intrigu betray...,3
4,157,5,aggress self glorif manipul whitewash,1
...,...,...,...,...
8524,155985,8540,will claustrophob concept,2
8525,155998,8541,despit annoy capabl clayburgh tambor great job...,2
8526,156022,8542,tri parodi genr joke unit state,1
8527,156032,8543,movi downfal substitut plot person,1


In [59]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn import metrics

In [103]:
print(aux.loc[2, "Phrase"])

fan ismail merchant work suspect hard time sit


In [107]:
aux = trainProcessed[trainProcessed['Phrase'].notnull()]
phrases = list(aux["Phrase"])
print(type(phrases), "\n", len(phrases), "\n", type(phrases[0]))

<class 'list'> 
 8497 
 <class 'str'>


In [122]:
vectorizer=CountVectorizer(min_df=1)
dtmTF = vectorizer.fit_transform(phrases)

In [113]:
dtmTF.shape

(8497, 10142)

In [120]:
df_tf = pd.DataFrame(dtmTF.toarray(), index=range(0, dtmTF.shape[0]), columns=vectorizer.get_feature_names())
df_tf.head(10)

/home/gfmco/.local/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aaa,aaliyah,aan,abagnal,abandon,abbass,abbott,abbrevi,abc,abderrahman,...,zish,ziyi,zoe,zombi,zone,zooland,zoom,zucker,zwick,zzzzzzzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [124]:
tfidf_transformer = TfidfTransformer()
dtmTF_IDF = tfidf_transformer.fit_transform(dtmTF)

In [127]:
df_tf_idf = pd.DataFrame(dtmTF_IDF.toarray(), index=range(0, dtmTF_IDF.shape[0]), columns=vectorizer.get_feature_names())
df_tf_idf.head(10)

/home/gfmco/.local/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aaa,aaliyah,aan,abagnal,abandon,abbass,abbott,abbrevi,abc,abderrahman,...,zish,ziyi,zoe,zombi,zone,zooland,zoom,zucker,zwick,zzzzzzzzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [143]:
lsa = TruncatedSVD(25, algorithm='randomized')
dtm_lsa_tf = lsa.fit_transform(dtmTF)
dtm_lsa_tf = Normalizer(copy=False).fit_transform(dtm_lsa_tf)

dtm_lsa_tf_idf = lsa.fit_transform(dtmTF_IDF)
dtm_lsa_tf_idf = Normalizer(copy=False).fit_transform(dtm_lsa_tf_idf)

In [144]:
print(dtm_lsa_tf.shape[0])
print(dtm_lsa_tf_idf.shape)

8497
(8497, 25)


In [145]:
df_lsa_tf = pd.DataFrame(dtm_lsa_tf, index=range(0, dtm_lsa_tf.shape[0]))
df_lsa_tf.head(10)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.177911,-0.019875,0.109136,0.820612,0.251568,0.407764,0.001004,-0.104768,-0.069970,0.003810,...,0.028716,-0.034586,-0.040154,-0.047065,-0.030476,0.025279,-0.030385,0.000763,0.006757,-0.001245
1,0.213610,-0.016495,0.110828,0.167998,-0.114257,-0.223056,0.069919,0.191720,0.098915,-0.028087,...,0.370408,0.143810,-0.023257,0.074652,0.400872,-0.114206,0.235133,-0.076787,-0.426064,-0.245071
2,0.143475,-0.042008,0.113868,0.226991,-0.166717,-0.499172,0.424463,-0.055147,0.479788,-0.239971,...,0.006587,-0.077477,0.078861,0.090799,0.049474,0.017578,-0.012397,-0.048931,0.118238,0.005439
3,0.363283,-0.049733,0.086713,0.261698,-0.426859,-0.143522,-0.122690,0.257328,0.150584,-0.193626,...,0.120074,0.193382,-0.164296,-0.040562,-0.124881,0.049409,-0.004014,0.033723,-0.267367,-0.302147
4,0.136953,-0.026661,0.114087,0.096064,-0.201390,-0.083686,-0.128592,0.135591,0.004323,-0.030996,...,-0.049857,0.158301,-0.124706,0.345279,-0.062355,0.220431,-0.298660,-0.150605,-0.035119,0.540940
5,0.136140,-0.014896,0.102517,0.189991,-0.326167,-0.322270,-0.529187,0.203669,-0.176638,0.093377,...,-0.096551,-0.187179,0.068980,-0.097825,0.116797,-0.054936,-0.042974,-0.070287,0.011772,-0.034171
6,0.364335,-0.042119,0.192905,0.209389,-0.389323,0.038749,-0.060499,0.183182,0.214237,-0.014279,...,0.059992,0.017902,-0.366152,0.037713,0.127656,0.261070,-0.169861,0.313307,0.293399,-0.069669
7,0.231620,-0.075654,0.875682,-0.248876,0.115643,0.071379,0.004144,0.012988,-0.080623,-0.185503,...,-0.042948,0.053710,-0.006433,-0.004331,-0.079375,-0.007876,0.022200,0.035783,-0.078656,0.003978
8,0.171880,-0.046824,0.142179,0.100601,-0.110821,-0.121836,-0.023479,-0.041890,0.062342,0.246425,...,-0.108289,0.115354,-0.249484,-0.161991,0.178235,-0.080995,-0.061402,0.657978,-0.227442,-0.344803
9,0.265881,-0.069151,0.154824,0.238703,-0.169699,-0.196651,-0.304024,0.194083,-0.029568,0.069213,...,0.065322,0.142851,-0.004004,0.286415,0.046829,-0.307054,-0.113767,0.378106,-0.256458,-0.114291


In [146]:
df_lsa_tf_idf = pd.DataFrame(dtm_lsa_tf_idf, index=range(0, dtm_lsa_tf_idf.shape[0]))
df_lsa_tf_idf.head(10)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.319660,0.128832,0.773997,0.174484,0.197442,-0.106473,-0.001786,-0.024976,-0.276506,-0.149212,...,-0.117037,0.013323,-0.092486,-0.171748,-0.072442,-0.011076,-0.013684,-0.033198,-0.012714,0.052935
1,0.121982,0.037358,-0.021134,0.055485,0.027722,-0.026578,0.097389,-0.028282,0.079117,0.065110,...,0.310541,0.173598,-0.206105,-0.448035,-0.425362,-0.126473,0.094311,-0.202303,0.098870,0.120522
2,0.239782,0.024116,-0.005551,0.205192,0.126839,-0.267797,0.287894,0.460443,0.574643,-0.073518,...,0.131236,0.105498,-0.055693,0.168679,0.049096,-0.029549,-0.097849,-0.043006,-0.030159,-0.004411
3,0.489934,0.123212,-0.178324,0.106723,0.141127,-0.193238,0.409141,-0.021739,-0.162741,0.140244,...,0.111903,-0.205657,0.090718,-0.177049,-0.034872,-0.035169,0.373818,0.012687,-0.031796,-0.122948
4,0.313634,0.069909,-0.168775,0.164930,0.086032,-0.111230,0.210159,-0.160606,-0.165533,-0.032982,...,-0.220456,-0.218270,0.176664,0.290833,0.243156,-0.222588,-0.195260,0.246376,-0.033827,0.187647
5,0.210459,0.081300,-0.079423,0.143583,0.234031,-0.078876,0.212027,-0.070327,-0.177642,0.525072,...,0.044883,0.064380,0.046774,-0.153232,-0.015304,-0.009262,-0.009262,-0.054222,0.047657,-0.058995
6,0.400643,0.129621,-0.100205,0.089099,0.061352,-0.070254,0.106325,-0.000818,-0.062571,0.040344,...,-0.116127,0.012205,-0.050056,0.056932,0.038372,-0.171499,0.083027,-0.194468,0.415603,0.587881
7,0.310773,0.030474,-0.231594,0.700231,-0.162253,0.314522,-0.200150,0.027828,-0.064859,-0.035823,...,0.021051,-0.032368,-0.049142,-0.132340,-0.017259,0.019991,-0.180471,0.010396,-0.074914,0.087297
8,0.221331,0.027170,-0.059304,0.210613,0.043472,-0.085755,0.033151,-0.040398,0.060247,0.023258,...,-0.222355,-0.534913,0.143633,0.053948,-0.169655,0.135728,0.191408,-0.104051,0.281897,-0.497111
9,0.281898,0.067279,0.068799,0.199949,0.057273,-0.129113,0.090804,-0.181503,-0.257473,0.157994,...,0.110211,0.247745,0.285294,-0.143470,0.028864,-0.404729,0.025426,-0.050163,0.216110,0.367167
